The ground Hatree-Fock state is when 1up and 1down orbitals (molecular orbitals) are occupied. This is (1, 1, 0, 0) in fermion occupation basis. In Bravyi-Kitaev basis, this is (1, 0, 0, 0), so just X(0). 

In [1]:
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.utils import hermitian_conjugated
from openfermion.utils import uccsd_generator
import numpy as np

from pyquil.paulis import PauliSum
from pyquil.api import WavefunctionSimulator
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli
from forestopenfermion import exponentiate

import numpy as np
import functools

from openfermion.hamiltonians import MolecularData, load_molecular_hamiltonian

In [2]:
from pyquil import get_qc
qc = get_qc('4q-qvm')

In [6]:
numQubit = 4
trials = 100
class Track:
    def __init__(self):
        self.cnt = 0
        
def build_ansatz(params):
    # Reference state
    a = None
    a = Program()

    single_amp = []
    double_amp = []

    # occupied: 0,1 unoccupied: 2,3
    single_amp.append([[2,0], params[0]])
    single_amp.append([[3,1], params[1]])

    if degree == 2:
        double_amp.append([[2,0,3,1], params[2]])

    ucc_gen = uccsd_generator(single_amp, double_amp)
    ucc_gen_qubitop = bravyi_kitaev(ucc_gen)
    try:
        ucc_program = exponentiate(ucc_gen_qubitop/(-1j))
    except:
        ucc_program = I(0)
    a += ucc_program
    return a

def solve_vqe_ucc_h2(hamiltonian: PauliSum, numLayer) -> float:
    # Construct a variational quantum eigensolver solution to find the lowest
    # eigenvalue of the given hamiltonian
    
    numParams = 0
    if degree == 1:
        numParams = 2
    elif degree == 2:
        numParams = 3
    params_init = np.random.rand(numParams)*0.5-0.25
    #params_init = [7.25536731e-07,  1.04666900e-06, -1.04874604e-01]
    
    track = Track()

    def inc(t, xk):
        t.cnt += 1

    def ansatz_energy(t, params):
        t.cnt += 1
        energy = 0
        ansatz = None
        ansatz = Program()
        ansatz += build_ansatz(params)
        
        energy = 0
        for term in hamiltonian:
            p = None
            p = Program()
            p += X(0)
            p += ansatz

            coeff = term.coefficient.real    # coefficient is real number in complex format
            opset =  term.operations_as_set()
            qubits_meas = []
            for op in opset:
                if op[1]=='X':
                    p += H(op[0])
                elif op[1]=='Y':
                    p += RZ(np.pi/2, op[0])
                    p += RY(np.pi/2, op[0])
                    p += RZ(-np.pi/2, op[0])
                elif op[1]=='Z':
                    pass
                qubits_meas.append(op[0])
  
            result = qc.run_and_measure(p, trials=trials)
            # measure the expectation value of eigenvalue in Z..Z basis. (number of 1 - number of -1)/trials
            eigval = 0
            for i in range(trials):
                tmp = 1
                for j in qubits_meas:
                    if result[j][i] == 1:
                        tmp = -tmp
                eigval += tmp
            energy += coeff*float(eigval)/trials
            
        print(energy)
        return energy

    option = {}
    option['ftol'] = 0.1
    params_answer = minimize(functools.partial(ansatz_energy, track), params_init, method='L-BFGS-B', options = option).x
    return ansatz_energy(track, params_answer), track.cnt

In [7]:
def get_ground_energy_ucc_h2(interaction_hamil, degree):
    fermionop_hamil = FermionOperator()
    for key in interaction_hamil:
        value = interaction_hamil[key]
        fermionop_hamil += FermionOperator(term=key, coefficient=value)
        
    qubitop_hamil = bravyi_kitaev(fermionop_hamil)
    pauliop_hamil = qubitop_to_pyquilpauli(qubitop_hamil)

    return solve_vqe_ucc_h2(pauliop_hamil, degree)


In [ ]:
basis = 'sto-3g'
multiplicity = 1  # 2S+1
charge = 0

import matplotlib.pyplot as plt
%matplotlib inline

bond_lengths = [0.7] #np.linspace(0.3, 2.5, 23)
vqe_ground_energies_mean = []
vqe_ground_energies_std = []
vqe_iter_mean = []
vqe_iter_std = []

for degree in [2]:
    gelist = []
    gestdlist = []
    iterlist = []
    iterstdlist = []
    for bond_length in bond_lengths:
        geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
        description = str(round(bond_length, 2))
        h2_interaction_hamil = load_molecular_hamiltonian(geometry,
            basis,
            multiplicity,
            description,
            n_active_electrons=None,
            n_active_orbitals=None)

        gelist_temp = []
        iterlist_temp = []
        for i in range(1):
            ge, cnt = get_ground_energy_ucc_h2(h2_interaction_hamil, degree)
            print('bond length: ', round(bond_length, 2), ' ground state energy: ', ge, ' iteration: ', cnt)
            gelist_temp.append(ge)
            iterlist_temp.append(cnt)
        gelist.append(np.mean(gelist_temp))
        gestdlist.append(np.std(gelist_temp))
        iterlist.append(np.mean(iterlist_temp))
        iterstdlist.append(np.std(iterlist_temp))
        
    vqe_ground_energies_mean.append(gelist)
    vqe_ground_energies_std.append(gestdlist)
    vqe_iter_mean.append(np.mean(iterlist))
    vqe_iter_std.append(np.mean(iterstdlist))


-1.0867964192664885
-1.1389734389962471
-1.0923418674108167
-1.1367111869804016
-0.3603139938386691
-0.38419792214766657
